In [1]:
import pandas as pd
import numpy as np

# 1. Statistical functions

## Percentage change

In [2]:
s = pd.Series([42, 57, 35, 101, 88])
s.pct_change() # 0.357143 == 35.7143%

0         NaN
1    0.357143
2   -0.385965
3    1.885714
4   -0.128713
dtype: float64

In [3]:
df = pd.DataFrame(np.random.randn(7, 4))
df.pct_change(periods=3)

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,-2.577846,-1.668189,-4.241796,-2.832929
4,-2.432966,-9.152440,-2.219185,-0.744716
5,3.557946,-4.417558,-1.112149,-1.949246
6,-1.114407,-9.060266,5.596633,-1.611418


## Covariance

In [4]:
s1 = pd.Series(np.random.randn(1000))
s2 = pd.Series(np.random.randn(1000))

s1.cov(s2)

-0.008060307386666795

In [5]:
# to compute pairwise covariances among the series(columns) in the DataFrame
df.cov()

,0,1,2,3
0,1.203516,-0.057326,-0.052582,0.043554
1,-0.057326,0.649635,-0.452778,0.361788
2,-0.052582,-0.452778,0.493335,-0.374563
3,0.043554,0.361788,-0.374563,0.891672


In [6]:
df[0].cov(df[1])

-0.05732627581880854

## Correlation

In [7]:
s1.corr(s2, method='pearson') # default

-0.008268366724711578

In [8]:
s1.corr(s2, method='kendall')

0.0067787787787787785

In [9]:
s1.corr(s2, method='spearman')

0.009465189465189466

In [10]:
# Pairwise correlation of DataFrame columns
df.corr()

,0,1,2,3
0,1.000000,-0.064833,-0.068241,0.042044
1,-0.064833,1.000000,-0.799797,0.475353
2,-0.068241,-0.799797,1.000000,-0.564744
3,0.042044,0.475353,-0.564744,1.000000


In [11]:
df1 = pd.DataFrame(np.random.randn(7, 4))
df2 = pd.DataFrame(np.random.randn(7, 4))

df1.corrwith(df2) #compute the correlation between the like-labeled series contained in df1 and df2

0    0.282301
1    0.622432
2   -0.054566
3    0.258843
dtype: float64

## Rank
The `rank()` method produces a data ranking with ties being assigned the mean of the ranks (by default) for the group.

In [12]:
s=pd.Series([2.2, 4.5, 3.8, 5.9, 3.8])
s.rank()

0    1.0
1    4.0
2    2.5
3    5.0
4    2.5
dtype: float64

In [13]:
df.rank()

,0,1,2,3
0,6.0,3.0,2.0,3.0
1,7.0,5.0,1.0,7.0
2,4.0,4.0,6.0,2.0
3,2.0,6.0,4.0,6.0
4,1.0,2.0,5.0,5.0
5,3.0,7.0,3.0,4.0
6,5.0,1.0,7.0,1.0
